In [1]:
# 安装必要的库
!pip install python-docx pyyaml pandas

In [2]:
# 导入所需的库
from docx import Document
import re
import yaml
import pandas as pd
from collections import defaultdict
import json

In [3]:
# 1. 文档预处理函数
def extract_word_content(doc_path):
    """从Word文档提取文本内容"""
    try:
        doc = Document(doc_path)
        full_text = []
        
        for paragraph in doc.paragraphs:
            if paragraph.text.strip():  # 忽略空行
                full_text.append(paragraph.text)
        
        return "\n".join(full_text)
    except Exception as e:
        print(f"读取Word文档时出错: {e}")
        return None

# 测试文档读取
doc_content = extract_word_content("AT_Commands.docx")  # 替换为您的文档路径
if doc_content:
    print(f"成功读取文档，共 {len(doc_content)} 字符")
    # 预览前500个字符
    print("\n文档预览:")
    print(doc_content[:500] + "...")
else:
    print("无法读取文档，请检查路径是否正确")

成功读取文档，共 3404 字符

文档预览:
通用AT指令
ATI–获取模组厂商信息
获取模组厂商信息，包括厂家、型号和版本。
命令格式
参数
示例
AT+GMR–查询版本信息
查询软件版本信息。
命令格式
参数
示例
AT+CSQ–获取信号强度
查询接收信号强度<rssi>。
命令格式
参数
示例
AT+CREG–查询网络注册状态
查询模组的当前网络注册状态。
命令格式
参数
示例
AT+CEREG–获取EPS网络注册状态
查询EPS网络注册状态。
命令格式
参数
示例
AT+COPS–网络选择
查询网络。
命令格式
参数
示例
AT+CIMI–查询国际移动用户识别码
获取国际移动用户识别码IMSI（international mobile subscriber identification）。
命令格式
参数
示例
AT+CGSN–获取通信模组IMEI号
获取模组的产品序列号，也就是IMEI号（International Mobile Equipment Identity）。
命令格式
参数
示例
AT+GSN–获取通信模组IMEI号
获取模组的产品序列号，也就是IMEI号（International Mobile Equip...


In [11]:
# 2. 定义解析模式和规则
# 在文档中识别这些模式
PATTERNS = {
    'command_start': r'^(\d+\.\d+\s+)?(AT[+\w]+)-',  # 命令开始，如"1.1 ATI-"
    'parameter_section': r'参数|参数\s*$',  # 参数部分开始
    'example_section': r'示例|示例\s*$',   # 示例部分开始
    'command_format': r'命令格式',        # 命令格式部分
}

# 提取命令描述的更精确模式
COMMAND_PATTERN = r'^(\d+\.\d+)\s+(AT[+\w]+)-([^。]+)'

# 参数提取模式
PARAM_PATTERN = r'<([^>]+)>([^<]+)'

In [19]:
# 同时支持破折号和冒号的解析规则
COMMAND_PATTERN = r'^(\d+\.\d+\s+)?(AT[+\w]+)[–\-：:]\s*([^。]+)'  # 匹配破折号、短横线、中文冒号和英文冒号

# 改进的解析函数 - 同时支持多种分隔符
def parse_at_commands(text_content):
    """解析AT命令文档 - 支持多种分隔符"""
    lines = text_content.split('\n')
    commands = []
    current_command = None
    current_section = None
    
    print("开始解析文档...")
    
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue
            
        # 调试：打印前20行处理过程
        if i < 20:
            print(f"处理行 {i}: {repr(line)}")
        
        # 检测命令开始 - 支持多种分隔符
        command_match = re.match(COMMAND_PATTERN, line)
        if command_match:
            if current_command:
                commands.append(current_command)
                print(f"已解析命令: {current_command['cmd']}")
            
            # 提取命令信息
            section = command_match.group(1)
            cmd = command_match.group(2)
            desc = command_match.group(3)
            
            current_command = {
                'section': section.strip() if section else None,
                'cmd': cmd,
                'desc': desc.strip(),
                'parameters': [],
                'examples': [],
                'format': None
            }
            current_section = None
            print(f"检测到命令: {cmd}")
            continue
        
        # 检测章节变化
        section_detected = False
        for section_type, patterns in SECTION_PATTERNS.items():
            for pattern in patterns:
                if re.match(pattern, line):
                    current_section = section_type
                    section_detected = True
                    if i < 20:  # 调试信息
                        print(f"检测到章节: {current_section}")
                    break
            if section_detected:
                break
                
        if section_detected:
            continue
            
        # 根据当前章节处理内容
        if current_command and current_section:
            if current_section == "parameters" and line:
                # 提取参数信息
                if ":" in line or "：" in line:
                    param_parts = re.split(r":|：", line, 1)
                    if len(param_parts) == 2:
                        param_name = param_parts[0].strip().strip('<>《》')
                        param_desc = param_parts[1].strip()
                        if param_name and param_desc:
                            current_command['parameters'].append({
                                'name': param_name,
                                'desc': param_desc
                            })
                elif re.search(r'<[^>]+>', line):
                    # 处理 <param>描述 格式
                    param_match = re.search(r'<([^>]+)>\s*([^<]*)', line)
                    if param_match:
                        param_name = param_match.group(1)
                        param_desc = param_match.group(2).strip()
                        if param_name and param_desc:
                            current_command['parameters'].append({
                                'name': param_name,
                                'desc': param_desc
                            })
            
            elif current_section == "examples" and line:
                # 跳过示例标题行
                if not any(keyword in line for keyword in ["示例", "例子", "example"]):
                    # 识别AT命令示例
                    if line.startswith("AT") or line.startswith("+") or line.startswith("OK") or line.startswith("ERROR"):
                        current_command['examples'].append(line)
            
            elif current_section == "format" and line:
                if not line.startswith("命令格式"):
                    current_command['format'] = line
    
    # 添加最后一个命令
    if current_command:
        commands.append(current_command)
        print(f"已解析命令: {current_command['cmd']}")
    
    print(f"\n解析完成，共提取 {len(commands)} 个AT命令")
    return commands

# 重新执行解析
if doc_content:
    commands = parse_at_commands(doc_content)
else:
    commands = []
    print("没有文档内容可供解析")

开始解析文档...
处理行 0: '通用AT指令'
处理行 1: 'ATI–获取模组厂商信息'
检测到命令: ATI
处理行 2: '获取模组厂商信息，包括厂家、型号和版本。'
处理行 3: '命令格式'
检测到章节: format
处理行 4: '参数'
检测到章节: parameters
处理行 5: '示例'
检测到章节: examples
处理行 6: 'AT+GMR–查询版本信息'
已解析命令: ATI
检测到命令: AT+GMR
处理行 7: '查询软件版本信息。'
处理行 8: '命令格式'
检测到章节: format
处理行 9: '参数'
检测到章节: parameters
处理行 10: '示例'
检测到章节: examples
处理行 11: 'AT+CSQ–获取信号强度'
已解析命令: AT+GMR
检测到命令: AT+CSQ
处理行 12: '查询接收信号强度<rssi>。'
处理行 13: '命令格式'
检测到章节: format
处理行 14: '参数'
检测到章节: parameters
处理行 15: '示例'
检测到章节: examples
处理行 16: 'AT+CREG–查询网络注册状态'
已解析命令: AT+CSQ
检测到命令: AT+CREG
处理行 17: '查询模组的当前网络注册状态。'
处理行 18: '命令格式'
检测到章节: format
处理行 19: '参数'
检测到章节: parameters
已解析命令: AT+CREG
检测到命令: AT+CEREG
已解析命令: AT+CEREG
检测到命令: AT+COPS
已解析命令: AT+COPS
检测到命令: AT+CIMI
已解析命令: AT+CIMI
检测到命令: AT+CGSN
已解析命令: AT+CGSN
检测到命令: AT+GSN
已解析命令: AT+GSN
检测到命令: AT+CCID
已解析命令: AT+CCID
检测到命令: AT+CGMM
已解析命令: AT+CGMM
检测到命令: AT+GMM
已解析命令: AT+GMM
检测到命令: AT+IPR
已解析命令: AT+IPR
检测到命令: AT+CFUN
已解析命令: AT+CFUN
检测到命令: AT+CMUX
已解析命令: AT+CMUX
检测到命令: AT+CCLK
已解析命令: 

In [34]:
# 直接提取方法 - 基于文档结构
def direct_extraction(text_content):
    """基于文档结构的直接提取方法"""
    lines = text_content.split('\n')
    commands = []
    current_command = None
    
    print("使用直接提取方法...")
    
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue
            
        # 检测命令开始
        if re.search(r'^(\d+\.\d+\s+)?AT[+\w]+[–\-：:]', line):
            if current_command:
                commands.append(current_command)
                
            # 提取命令和描述
            parts = re.split(r'[–\-：:]', line, 1)
            if len(parts) == 2:
                cmd_part = parts[0].strip()
                desc = parts[1].strip()
                
                # 提取命令部分中的AT命令
                cmd_match = re.search(r'(AT[+\w]+)', cmd_part)
                if cmd_match:
                    cmd = cmd_match.group(1)
                    
                    # 提取节号
                    section_match = re.search(r'(\d+\.\d+)', cmd_part)
                    section = section_match.group(1) if section_match else None
                    
                    current_command = {
                        'section': section,
                        'cmd': cmd,
                        'desc': desc,
                        'parameters': [],
                        'examples': [],
                        'format': None
                    }
                    print(f"找到命令: {cmd}")
        
        # 检测参数部分
        elif line == "参数" and current_command:
            # 接下来的几行可能是参数
            param_lines = []
            for j in range(i+1, min(i+10, len(lines))):  # 查看接下来的10行
                next_line = lines[j].strip()
                if not next_line or next_line in ["示例", "命令格式", "AT"]:
                    break
                param_lines.append(next_line)
            
            # 处理参数行
            for param_line in param_lines:
                # 尝试提取参数
                if "<" in param_line and ">" in param_line:
                    param_match = re.search(r'<([^>]+)>\s*([^<]*)', param_line)
                    if param_match:
                        param_name = param_match.group(1)
                        param_desc = param_match.group(2).strip()
                        if param_name and param_desc:
                            current_command['parameters'].append({
                                'name': param_name,
                                'desc': param_desc
                            })
                            print(f"提取参数: {param_name}")
        
        # 检测示例部分
        elif line == "示例" and current_command:
            # 接下来的几行可能是示例
            example_lines = []
            for j in range(i+1, min(i+10, len(lines))):  # 查看接下来的10行
                next_line = lines[j].strip()
                if not next_line or next_line in ["参数", "命令格式", "AT"]:
                    break
                example_lines.append(next_line)
            
            # 处理示例行
            for example_line in example_lines:
                if example_line and not example_line.startswith("示例"):
                    current_command['examples'].append(example_line)
                    print(f"提取示例: {example_line}")
        
        # 检测格式部分
        elif line == "命令格式" and current_command:
            # 下一行可能是格式
            if i+1 < len(lines):
                format_line = lines[i+1].strip()
                if format_line and not format_line.startswith("命令格式"):
                    current_command['format'] = format_line
                    print(f"提取格式: {format_line}")
    
    if current_command:
        commands.append(current_command)
    
    print(f"共找到 {len(commands)} 个AT命令")
    return commands

# 使用直接提取方法
if doc_content:
    commands = direct_extraction(doc_content)
else:
    commands = []
    print("没有文档内容可供解析")

使用直接提取方法...
找到命令: ATI
提取格式: 参数
提取示例: AT+GMR–查询版本信息
提取示例: 查询软件版本信息。
找到命令: AT+GMR
提取格式: 参数
提取示例: AT+CSQ–获取信号强度
提取示例: 查询接收信号强度<rssi>。
找到命令: AT+CSQ
提取格式: 参数
提取示例: AT+CREG–查询网络注册状态
提取示例: 查询模组的当前网络注册状态。
找到命令: AT+CREG
提取格式: 参数
提取示例: AT+CEREG–获取EPS网络注册状态
提取示例: 查询EPS网络注册状态。
找到命令: AT+CEREG
提取格式: 参数
提取示例: AT+COPS–网络选择
提取示例: 查询网络。
找到命令: AT+COPS
提取格式: 参数
提取示例: AT+CIMI–查询国际移动用户识别码
提取示例: 获取国际移动用户识别码IMSI（international mobile subscriber identification）。
找到命令: AT+CIMI
提取格式: 参数
提取示例: AT+CGSN–获取通信模组IMEI号
提取示例: 获取模组的产品序列号，也就是IMEI号（International Mobile Equipment Identity）。
找到命令: AT+CGSN
提取格式: 参数
提取示例: AT+GSN–获取通信模组IMEI号
提取示例: 获取模组的产品序列号，也就是IMEI号（International Mobile Equipment Identity）。
找到命令: AT+GSN
提取格式: 参数
提取示例: AT+CCID–获取SIM卡标识
提取示例: 获取SIM卡的ICCID。
找到命令: AT+CCID
提取格式: 参数
提取示例: AT+CGMM–查询模组型号
提取示例: 查询模组型号。
找到命令: AT+CGMM
提取格式: 参数
提取示例: AT+GMM–查询模组型号
提取示例: 查询模组型号。
找到命令: AT+GMM
提取格式: 参数
提取示例: AT+IPR–设置模组波特率
提取示例: 设置模组波特率，默认掉电保存。
提取示例: 若波特率查询返回为0，表示模组波特率自适应。默认为波特率自适应（备注：自适应波特率不超过115200）。
找到命令: AT+IPR
提取格式: 参数


C:\Users\txiab\AppData\Local\Temp\ipykernel_31660\3507291280.py:21: DeprecationWarning: 'maxsplit' is passed as positional argument
  parts = re.split(r'[–\-：:]', line, 1)


In [20]:
# 诊断代码：查看文档前50行内容
print("文档前50行内容：")
lines = doc_content.split('\n')
for i, line in enumerate(lines[:50]):
    print(f"{i:2d}: {repr(line)}")  # 使用repr()显示原始格式，包括空格和特殊字符

# 检查是否有AT命令相关的行
print("\n查找包含'AT'的行：")
at_lines = [line for line in lines if 'AT' in line]
for i, line in enumerate(at_lines[:20]):
    print(f"{i:2d}: {repr(line)}")

文档前50行内容：
 0: '通用AT指令'
 1: 'ATI–获取模组厂商信息'
 2: '获取模组厂商信息，包括厂家、型号和版本。'
 3: '命令格式'
 4: '参数'
 5: '示例'
 6: 'AT+GMR–查询版本信息'
 7: '查询软件版本信息。'
 8: '命令格式'
 9: '参数'
10: '示例'
11: 'AT+CSQ–获取信号强度'
12: '查询接收信号强度<rssi>。'
13: '命令格式'
14: '参数'
15: '示例'
16: 'AT+CREG–查询网络注册状态'
17: '查询模组的当前网络注册状态。'
18: '命令格式'
19: '参数'
20: '示例'
21: 'AT+CEREG–获取EPS网络注册状态'
22: '查询EPS网络注册状态。'
23: '命令格式'
24: '参数'
25: '示例'
26: 'AT+COPS–网络选择'
27: '查询网络。'
28: '命令格式'
29: '参数'
30: '示例'
31: 'AT+CIMI–查询国际移动用户识别码'
32: '获取国际移动用户识别码IMSI（international mobile subscriber identification）。'
33: '命令格式'
34: '参数'
35: '示例'
36: 'AT+CGSN–获取通信模组IMEI号'
37: '获取模组的产品序列号，也就是IMEI号（International Mobile Equipment Identity）。'
38: '命令格式'
39: '参数'
40: '示例'
41: 'AT+GSN–获取通信模组IMEI号'
42: '获取模组的产品序列号，也就是IMEI号（International Mobile Equipment Identity）。'
43: '命令格式'
44: '参数'
45: '示例'
46: 'AT+CCID–获取SIM卡标识'
47: '获取SIM卡的ICCID。'
48: '命令格式'
49: '参数'

查找包含'AT'的行：
 0: '通用AT指令'
 1: 'ATI–获取模组厂商信息'
 2: 'AT+GMR–查询版本信息'
 3: 'AT+CSQ–获取信号强度'
 4: 'AT+CREG–查询网络注册状态'
 5: 'AT+CEREG–获取

In [28]:
# 执行解析
if doc_content:
    commands = parse_at_commands(doc_content)
else:
    commands = []
    print("没有文档内容可供解析")

开始解析文档...
处理行 0: '通用AT指令'
处理行 1: 'ATI–获取模组厂商信息'
检测到命令: ATI
处理行 2: '获取模组厂商信息，包括厂家、型号和版本。'
处理行 3: '命令格式'
检测到章节: format
处理行 4: '参数'
检测到章节: parameters
处理行 5: '示例'
检测到章节: examples
处理行 6: 'AT+GMR–查询版本信息'
已解析命令: ATI
检测到命令: AT+GMR
处理行 7: '查询软件版本信息。'
处理行 8: '命令格式'
检测到章节: format
处理行 9: '参数'
检测到章节: parameters
处理行 10: '示例'
检测到章节: examples
处理行 11: 'AT+CSQ–获取信号强度'
已解析命令: AT+GMR
检测到命令: AT+CSQ
处理行 12: '查询接收信号强度<rssi>。'
处理行 13: '命令格式'
检测到章节: format
处理行 14: '参数'
检测到章节: parameters
处理行 15: '示例'
检测到章节: examples
处理行 16: 'AT+CREG–查询网络注册状态'
已解析命令: AT+CSQ
检测到命令: AT+CREG
处理行 17: '查询模组的当前网络注册状态。'
处理行 18: '命令格式'
检测到章节: format
处理行 19: '参数'
检测到章节: parameters
已解析命令: AT+CREG
检测到命令: AT+CEREG
已解析命令: AT+CEREG
检测到命令: AT+COPS
已解析命令: AT+COPS
检测到命令: AT+CIMI
已解析命令: AT+CIMI
检测到命令: AT+CGSN
已解析命令: AT+CGSN
检测到命令: AT+GSN
已解析命令: AT+GSN
检测到命令: AT+CCID
已解析命令: AT+CCID
检测到命令: AT+CGMM
已解析命令: AT+CGMM
检测到命令: AT+GMM
已解析命令: AT+GMM
检测到命令: AT+IPR
已解析命令: AT+IPR
检测到命令: AT+CFUN
已解析命令: AT+CFUN
检测到命令: AT+CMUX
已解析命令: AT+CMUX
检测到命令: AT+CCLK
已解析命令: 

In [36]:
# 手动填充缺失数据
def manually_fill_missing_data(commands):
    """手动填充缺失的数据"""
    print("手动填充缺失数据...")
    
    # 为每个命令手动添加缺失的数据
    for cmd in commands:
        cmd_name = cmd.get('cmd', '未知命令')
        
        # 根据命令名称添加特定数据
        if cmd_name == "ATI":
            if not cmd.get('parameters'):
                cmd['parameters'] = [
                    {'name': 'manufacturer', 'desc': '模组厂商信息、产品名称、版本号'},
                    {'name': 'module_version', 'desc': '模组型号'},
                    {'name': 'soft_version', 'desc': '模组软件版本'}
                ]
            if not cmd.get('examples'):
                cmd['examples'] = ['ATI', '查询厂商信息', '+CGMI: Neoway', 'OK']
            if not cmd.get('format'):
                cmd['format'] = 'ATI'
        
        elif cmd_name == "AT+GMR":
            if not cmd.get('parameters'):
                cmd['parameters'] = [
                    {'name': 're version', 'desc': '模组软件版本信息'}
                ]
            if not cmd.get('examples'):
                cmd['examples'] = ['AT+GMR', '+GMR:N706-R004-STD-B2-003', 'OK']
            if not cmd.get('format'):
                cmd['format'] = 'AT+GMR'
        
        elif cmd_name == "AT+CSQ":
            if not cmd.get('parameters'):
                cmd['parameters'] = [
                    {'name': 'signal', 'desc': '信号强度'},
                    {'name': 'ber', 'desc': '误码率'}
                ]
            if not cmd.get('examples'):
                cmd['examples'] = ['AT+CSQ', '+CSQ:19,2', 'OK']
            if not cmd.get('format'):
                cmd['format'] = 'AT+CSQ'
        
        # 添加更多命令的特定数据...
        
        # 为所有命令添加默认格式（如果缺失）
        if not cmd.get('format'):
            cmd['format'] = cmd.get('cmd', '未知命令')
    
    print("手动填充完成")
    return commands

# 使用手动填充
if commands:
    commands = manually_fill_missing_data(commands)
else:
    print("没有命令数据可供填充")

手动填充缺失数据...
手动填充完成


In [38]:
# 4. 数据导出函数
def commands_to_csv(commands, output_path):
    """将命令数据导出为CSV"""
    # 准备CSV数据
    csv_data = []
    for cmd in commands:
        # 将参数列表转换为字符串
        params_str = "; ".join([f"{p['name']}: {p['desc']}" for p in cmd['parameters']])
        # 将示例列表转换为字符串
        examples_str = "; ".join(cmd['examples'])
        
        csv_data.append({
            'section': cmd.get('section', ''),
            'command': cmd.get('cmd', ''),
            'description': cmd.get('desc', ''),
            'parameters': params_str,
            'examples': examples_str,
            'format': cmd.get('format', '')
        })
    
    # 创建DataFrame并保存
    df = pd.DataFrame(csv_data)
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"CSV文件已保存: {output_path}")
    return df

def commands_to_yaml(commands, output_path):
    """将命令数据导出为YAML"""
    structured_data = {
        'at_commands': commands
    }
    
    with open(output_path, 'w', encoding='utf-8') as yamlfile:
        yaml.dump(structured_data, yamlfile, allow_unicode=True, sort_keys=False, default_flow_style=False)
    
    print(f"YAML文件已保存: {output_path}")
    
    # 预览YAML内容
    with open(output_path, 'r', encoding='utf-8') as f:
        content = f.read()
        print("\nYAML文件预览:")
        print(content[:500] + "..." if len(content) > 500 else content)

In [39]:
# 5. 执行转换和导出
if commands:
    # 导出为CSV（中间格式）
    csv_df = commands_to_csv(commands, "at_commands.csv")
    
    # 显示CSV预览
    print("\nCSV数据预览:")
    display(csv_df.head())
    
    # 导出为YAML
    commands_to_yaml(commands, "at_commands.yaml")
    
    # 可选：保存为JSON格式
    with open("at_commands.json", "w", encoding="utf-8") as f:
        json.dump({"at_commands": commands}, f, ensure_ascii=False, indent=2)
    print("JSON文件已保存: at_commands.json")
else:
    print("没有命令数据可供导出")

CSV文件已保存: at_commands.csv

CSV数据预览:


,section,command,description,parameters,examples,format
0,None,ATI,获取模组厂商信息,manufacturer: 模组厂商信息、产品名称、版本号; module_version:...,AT+GMR–查询版本信息; 查询软件版本信息。,参数
1,None,AT+GMR,查询版本信息,re version: 模组软件版本信息,AT+CSQ–获取信号强度; 查询接收信号强度<rssi>。,参数
2,None,AT+CSQ,获取信号强度,signal: 信号强度; ber: 误码率,AT+CREG–查询网络注册状态; 查询模组的当前网络注册状态。,参数
3,None,AT+CREG,查询网络注册状态,,AT+CEREG–获取EPS网络注册状态; 查询EPS网络注册状态。,参数
4,None,AT+CEREG,获取EPS网络注册状态,,AT+COPS–网络选择; 查询网络。,参数


YAML文件已保存: at_commands.yaml

YAML文件预览:
at_commands:
- section: null
  cmd: ATI
  desc: 获取模组厂商信息
  parameters:
  - name: manufacturer
    desc: 模组厂商信息、产品名称、版本号
  - name: module_version
    desc: 模组型号
  - name: soft_version
    desc: 模组软件版本
  examples:
  - AT+GMR–查询版本信息
  - 查询软件版本信息。
  format: 参数
- section: null
  cmd: AT+GMR
  desc: 查询版本信息
  parameters:
  - name: re version
    desc: 模组软件版本信息
  examples:
  - AT+CSQ–获取信号强度
  - 查询接收信号强度<rssi>。
  format: 参数
- section: null
  cmd: AT+CSQ
  desc: 获取信号强度
  parameters:
  - name: signal
    d...
JSON文件已保存: at_commands.json


In [40]:
# 6. 数据验证和清理函数
def validate_commands(commands):
    """验证命令数据的完整性"""
    print("开始验证命令数据...")
    
    issues = []
    for i, cmd in enumerate(commands):
        # 检查必要字段
        if not cmd.get('cmd'):
            issues.append(f"命令 #{i}: 缺少命令名称")
        if not cmd.get('desc'):
            issues.append(f"命令 #{i} {cmd.get('cmd', '未知')}: 缺少描述")
    
    # 检查重复命令
    cmd_names = [cmd.get('cmd', '') for cmd in commands]
    duplicates = set([x for x in cmd_names if cmd_names.count(x) > 1])
    if duplicates:
        issues.append(f"发现重复命令: {', '.join(duplicates)}")
    
    if issues:
        print(f"发现 {len(issues)} 个问题:")
        for issue in issues:
            print(f"  - {issue}")
    else:
        print("数据验证通过，未发现问题")
    
    return issues

# 执行验证
if commands:
    issues = validate_commands(commands)
else:
    print("没有命令数据可供验证")

开始验证命令数据...
数据验证通过，未发现问题


In [41]:
# 7. 手动修正辅助函数
def manual_correction_helper(commands):
    """辅助手动修正数据的函数"""
    print("手动修正辅助工具")
    print("=" * 50)
    
    for i, cmd in enumerate(commands):
        print(f"{i+1}. {cmd.get('cmd', '未知命令')}: {cmd.get('desc', '无描述')}")
        print(f"   参数: {len(cmd.get('parameters', []))} 个")
        print(f"   示例: {len(cmd.get('examples', []))} 个")
        print()
    
    print("使用 commands[索引] 访问和修改特定命令")
    print("例如: commands[0]['desc'] = '新的描述'")
    
    return commands

# 启动手动修正辅助
if commands:
    commands = manual_correction_helper(commands)
else:
    print("没有命令数据可供修正")

手动修正辅助工具
1. ATI: 获取模组厂商信息
   参数: 3 个
   示例: 2 个

2. AT+GMR: 查询版本信息
   参数: 1 个
   示例: 2 个

3. AT+CSQ: 获取信号强度
   参数: 2 个
   示例: 2 个

4. AT+CREG: 查询网络注册状态
   参数: 0 个
   示例: 2 个

5. AT+CEREG: 获取EPS网络注册状态
   参数: 0 个
   示例: 2 个

6. AT+COPS: 网络选择
   参数: 0 个
   示例: 2 个

7. AT+CIMI: 查询国际移动用户识别码
   参数: 0 个
   示例: 2 个

8. AT+CGSN: 获取通信模组IMEI号
   参数: 0 个
   示例: 2 个

9. AT+GSN: 获取通信模组IMEI号
   参数: 0 个
   示例: 2 个

10. AT+CCID: 获取SIM卡标识
   参数: 0 个
   示例: 2 个

11. AT+CGMM: 查询模组型号
   参数: 0 个
   示例: 2 个

12. AT+GMM: 查询模组型号
   参数: 0 个
   示例: 3 个

13. AT+IPR: 设置模组波特率
   参数: 0 个
   示例: 3 个

14. AT+CFUN: 设置模组功能
   参数: 0 个
   示例: 4 个

15. AT+CMUX: 串口多路复用指令
   参数: 0 个
   示例: 3 个

16. AT+CCLK: 时钟管理
   参数: 0 个
   示例: 3 个

17. AT+CPIN: 输入PIN码
   参数: 0 个
   示例: 2 个

18. AT+CLCK: PIN使能与查询功能指令
   参数: 0 个
   示例: 3 个

19. AT+CPWD: 修改密码指令
   参数: 0 个
   示例: 2 个

20. AT+CGDCONT: 设置PDP格式
   参数: 0 个
   示例: 6 个

21. AT+XGAUTH: 用户认证
   参数: 0 个
   示例: 6 个

22. AT+CGATT: 设置GPRS附着和分离
   参数: 0 个
   示例: 9 个

23. AT+CESQ: 扩展信号强度
 

In [25]:
# 8. 最终导出和总结
if commands:
    # 最终导出
    commands_to_yaml(commands, "at_commands_final.yaml")
    
    # 生成转换报告
    total_commands = len(commands)
    total_params = sum(len(cmd.get('parameters', [])) for cmd in commands)
    total_examples = sum(len(cmd.get('examples', [])) for cmd in commands)
    
    print("=" * 60)
    print("转换完成报告")
    print("=" * 60)
    print(f"成功转换的命令数量: {total_commands}")
    print(f"提取的参数总数: {total_params}")
    print(f"提取的示例总数: {total_examples}")
    print(f"最终YAML文件: at_commands_final.yaml")
    print(f"中间CSV文件: at_commands.csv")
    print("=" * 60)
    
    # 显示前几个命令的详情
    print("\n前3个命令的详情:")
    for i, cmd in enumerate(commands[:3]):
        print(f"{i+1}. {cmd['cmd']}: {cmd['desc']}")
        if cmd.get('parameters'):
            print("   参数:")
            for param in cmd['parameters']:
                print(f"     - {param['name']}: {param['desc']}")
        if cmd.get('examples'):
            print("   示例:")
            for example in cmd['examples'][:2]:  # 只显示前2个示例
                print(f"     - {example}")
        print()
else:
    print("没有命令数据可供导出")

YAML文件已保存: at_commands_final.yaml

YAML文件预览:
at_commands:
- section: null
  cmd: ATI
  desc: 获取模组厂商信息
  parameters: []
  examples: []
  format: null
- section: null
  cmd: AT+GMR
  desc: 查询版本信息
  parameters: []
  examples: []
  format: null
- section: null
  cmd: AT+CSQ
  desc: 获取信号强度
  parameters: []
  examples: []
  format: null
- section: null
  cmd: AT+CREG
  desc: 查询网络注册状态
  parameters: []
  examples: []
  format: null
- section: null
  cmd: AT+CEREG
  desc: 获取EPS网络注册状态
  parameters: []
  examples: []
  format: null
- section: null
  ...
转换完成报告
成功转换的命令数量: 42
提取的参数总数: 0
提取的示例总数: 2
最终YAML文件: at_commands_final.yaml
中间CSV文件: at_commands.csv

前3个命令的详情:
1. ATI: 获取模组厂商信息

2. AT+GMR: 查询版本信息

3. AT+CSQ: 获取信号强度

